In [1]:
import torch

In [2]:
# ! pip install -U sentence-transformers

In [10]:
from loaders.elco_dataloader import get_loaders
train_loader, test_loader = get_loaders("data/ELCo.csv", batch_size=1, shuffle=True, num_workers=0)

In [4]:
iter(train_loader).next()

[{0: [('national park',), ('national park',)],
  1: [('TOP arrow',), ('national park',), ('TOP arrow national park',)],
  2: [('up arrow',), ('national park',), ('up arrow national park',)],
  3: [('TOP arrow',),
   ('water wave',),
   ('national park',),
   ('TOP arrow water wave national park',)],
  4: [('national park',), ('national park',)],
  5: [('up arrow',), ('national park',), ('up arrow national park',)],
  6: [('ladder',), ('national park',), ('ladder national park',)],
  7: [('water wave',),
   ('backhand index pointing up',),
   ('water wave backhand index pointing up',)]},
 ('high river',)]

In [12]:
value = iter(train_loader).next()
from itertools import chain
print(value[0][0])
list(chain(*value[0][0]))

[('cross mark',), ('page facing up',), ('cross mark page facing up',)]


['cross mark', 'page facing up', 'cross mark page facing up']

In [13]:
iter(test_loader).next()

[{0: [('sun',),
   ('hot springs',),
   ('hot face',),
   ('thermometer',),
   ('sun hot springs hot face thermometer',)],
  1: [('TOP arrow',), ('hot face',), ('TOP arrow hot face',)],
  2: [('thermometer',),
   ('face with thermometer',),
   ('thermometer face with thermometer',)],
  3: [('face with thermometer',),
   ('woozy face',),
   ('hot face',),
   ('face with thermometer woozy face hot face',)],
  4: [('hot face',),
   ('hot springs',),
   ('sun',),
   ('bed',),
   ('hot face hot springs sun bed',)],
  5: [('thermometer',), ('sun',), ('thermometer sun',)],
  6: [('face with thermometer',), ('face with thermometer',)],
  7: [('thermometer',),
   ('face with thermometer',),
   ('backhand index pointing up',),
   ('thermometer face with thermometer backhand index pointing up',)]},
 ('high temperature',)]

In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
x, y = iter(train_loader).next()
# flatten list of lists
print(x[0])
x = list(chain(*x[0]))
print(x)
x = model.encode(x)
y = model.encode(y[0])
x = x.mean(axis=0) # Going with a simple mean for now
x.shape, y.shape

[('hot springs',), ('foot',), ('leg',), ('nose',), ('ear',), ('hand with fingers splayed',), ('hot springs foot leg nose ear hand with fingers splayed',)]
['hot springs', 'foot', 'leg', 'nose', 'ear', 'hand with fingers splayed', 'hot springs foot leg nose ear hand with fingers splayed']


((384,), (384,))

In [15]:
import numpy as np 

class SimilarityMetrics:
    def __init__(self, similarity_metric):
        self.similarity_metric = similarity_metric
    
    def type_check(self, embedding):
        if isinstance(embedding, list):
            embedding = torch.stack(embedding)
        if isinstance(embedding, np.ndarray):
            embedding = torch.from_numpy(embedding)
        return embedding
    
    def __call__(self, x_embedding, y_embedding):
        return self.similarity_metric(x_embedding, y_embedding)

class CosineSimilarity(SimilarityMetrics):
    def __init__(self):
        super().__init__(torch.nn.CosineSimilarity(dim=0))
    
    def __call__(self, x_embedding, y_embedding):
        x_embedding = self.type_check(x_embedding)
        y_embedding = self.type_check(y_embedding)
        
        if x_embedding.ndim == 1:
            x_embedding = x_embedding.view(-1, 1)
        if y_embedding.ndim == 1:
            y_embedding = y_embedding.view(-1, 1)
        return super().__call__(x_embedding, y_embedding)

class EucledianDistance(SimilarityMetrics):
    def __init__(self):
        super().__init__(torch.nn.PairwiseDistance(p=2))
    
    def __call__(self, x_embedding, y_embedding):
        x_embedding = self.type_check(x_embedding)
        y_embedding = self.type_check(y_embedding)
        
        if x_embedding.ndim == 1:
            x_embedding = x_embedding.unsqueeze(0)
        if y_embedding.ndim == 1:
            y_embedding = y_embedding.unsqueeze(0)
        return -super().__call__(x_embedding, y_embedding)


In [16]:
CosineSimilarity()(x, y)

tensor([0.5228])

In [17]:
torch.tensor(x).unsqueeze(0).shape


torch.Size([1, 384])

In [18]:
EucledianDistance()(x, y)

tensor([-0.8624])

In [19]:
import torch.nn as nn
from sentence_transformers import SentenceTransformer


class TeacherModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = SentenceTransformer('bert-base-nli-mean-tokens')

    def forward(self, x):
        return self.model.encode(x)

class StudentModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = SentenceTransformer('bert-base-nli-mean-tokens')

    def forward(self, x, y):
        return self.model.encode(x), self.model.encode(y)

class ELCoM(nn.Module):
    def __init__(self):
        super().__init__()
        self.teacher = TeacherModel()
        self.student = StudentModel()
    
    def forward(self, x, y):
        teacher_en = self.teacher(x)
        student_en, student_em = self.student(x, y)
        return teacher_en, student_en, student_em

In [20]:
x, y = iter(train_loader).next()
x[0]

[('shorts',), ('alien',), ('shorts alien',)]

In [21]:
model = ELCoM()
teacher_en, student_en, student_em = model(x, y)
teacher_en.shape, student_en.shape, student_em.shape

((8, 768), (8, 768), (1, 768))

In [22]:
from models.sbert import SBERT

model = SBERT()
model(x)

array([[ 0.30831382,  0.3514907 ,  0.6976047 , ...,  0.53146845,
         0.8605992 ,  0.08637048],
       [ 0.15562136,  0.8941867 ,  0.88533294, ...,  0.4209026 ,
         0.46350625,  0.20158474],
       [-0.01096022,  0.760641  ,  1.3321491 , ..., -0.32805952,
         0.69355524, -0.15827833],
       ...,
       [-0.5131449 ,  0.40606   ,  0.7688541 , ...,  0.19555578,
         0.533955  ,  0.12900162],
       [ 0.10136722,  0.06409666,  1.3900058 , ...,  0.07869086,
         0.5670041 , -0.19467375],
       [ 0.12369446,  0.3582304 ,  1.2054628 , ...,  0.38293025,
         0.90999806,  0.0404312 ]], dtype=float32)

In [23]:
from metrics.accuracy_metrics import AccuracyMetrics
from metrics.similarity_metrics import CosineSimilarity
from tqdm import tqdm

class UnsupervisedBaselineTrainer:
    def __init__(self, model, data_loader, similarity_metric, accuracy_metric):
        self.model = model
        self.data_loader = data_loader
        self.similarity_metric = similarity_metric
        self.accuracy_metric = accuracy_metric
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def train(self):
        """
        The baseline isn't supposed to be trained, so this method is empty.
        """
        pass

    def test(self):
        accuracy = 0
        for x, y in tqdm(self.data_loader):
            y = y[0]
            rank_scores = []
            y_score = self.model(y)
            for rank, sentences in x.items():
                sentences = list(chain(*sentences))
                embeddings = self.model(sentences).mean(axis=0) # simple mean
                rank_scores.append((self.similarity_metric(embeddings, y_score).cpu().detach().numpy()[0], rank))
            sorted_rank_scores = sorted(rank_scores, key=lambda x: x[0], reverse=True)
            accuracy += self.accuracy_metric(sorted_rank_scores)
        return accuracy / len(self.data_loader)

In [24]:
trainer = UnsupervisedBaselineTrainer(model, test_loader, CosineSimilarity(), AccuracyMetrics().accuracy)
acc = trainer.test()
print(f"Accuracy: {acc * 100:.2f}%")

100%|██████████| 42/42 [01:11<00:00,  1.71s/it]

Accuracy: 13.96%
